In [1]:
import torch
print(torch.cuda.device_count())
from unsloth import FastLanguageModel

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, BitsAndBytesConfig
import transformers
from datasets import load_dataset, Dataset
import pprint
import json
from pathlib import Path
import os
import numpy as np
import time
import re
import pandas as pd

2
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
device = f"cuda"
from huggingface_hub import hf_hub_download
import os


if torch.cuda.get_device_capability()[0] >= 8:
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16
# attn_implementation = "eager"
print(f"Using {attn_implementation} for attention computation.")

# ------------------------------------------------------------

model_id = 'sh2orc/Llama-3.1-Korean-8B-Instruct'
# model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
# model_id = 'Saxo/Linkbricks-Horizon-AI-Korean-Gemma-2-sft-dpo-27B'
# model_id = 'Bllossom/llama-3-Korean-Bllossom-70B'

model_dir = f"/model/{model_id.replace('/', '-')}"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    # max_seq_length = max_seq_length,
    dtype = torch_dtype,
    load_in_4bit = False,
    load_in_8bit = False,
    # quantization_config=BitsAndBytesConfig(
    #     load_in_4bit=True,
    #     bnb_4bit_use_double_quant=True,
    #     bnb_4bit_quant_type="nf4",
    #     bnb_4bit_compute_dtype=torch_dtype
    #     # load_in_8bit=True,
    #     # llm_int8_enable_fp32_cpu_offload=False if not "27B" in model_id else True,
    # ),
    # device_map=device,
    cache_dir=f"{model_dir}/cache",
    attn_implementation=attn_implementation,
    local_files_only=True
)
FastLanguageModel.for_inference(model)
tokenizer.padding_side = "left"

Using flash_attention_2 for attention computation.
==((====))==  Unsloth 2025.2.5: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.689 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [3]:
from pathlib import Path  # Import Path from pathlib
import json
dataset_name = "v7-250309-reduceinputanddatefunctioncall"
dataset_dir = Path(f"/workspace/finetuning/dataset/{dataset_name}")

metadatas = {}
for directory in dataset_dir.iterdir():
    if directory.is_dir() and "scenario" in directory.name:
        with open(f"{directory}/metadata.json", "r", encoding="utf-8") as f:
            metadatas[directory.name] = json.loads(f.read())

print(metadatas)
        

{'scenario1': {'site_name': 'YongDongIllHighSchool', 'user_name': '홍길동', 'user_role': 'customer', 'idu_name': '01_IB5', 'idu_mapping': {'01_IB5': ['우리반'], '01_IB7': ['옆반'], '02_I81': ['앞반']}, 'modality_mapping': {'roomtemp': ['실내온도'], 'settemp': ['설정온도'], 'oper': ['전원']}, 'current_datetime': '2022-09-30 12:00:00'}}


In [4]:
file_name = "db_gt_v7.json"
file_path = f"/workspace/experiments/{file_name}"

with open(file_path, "r") as f:
    datas = json.load(f)


In [1]:
import time
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
system_prompt = """
너는 질문에 대해 주어진 결과값을 참고해 정직하고 자연스러운 답변을 제공하는 인공지능 모델이다.
숫자는 소숫점 2자리까지 표시해.
만약 결과의 특정 변수가 공란/None이거나 특정 변수의 element 개수가 10개보다 많다면, 해당 변수는 건너뛰어도 무방해.
이 외의 경우에는 결과를 왜곡하지 말고 정직하게 답해.
예시를 참고해 최대한 모든 변수를 다루도록 노력해.
예시를 참고하지만 꼭 같을 필욘 없어.
답변이 여러개라면 모든 날짜를 나열하지 않고, 대표적인 날짜를 몇 개 제시하여 불필요한 긴 목록을 줄여줘.
없는 정보에 대해서는 추측을 하는 답변 조차 하지 말고, HVAC 시스템과 관련 없는 질문(농담 등)에 대해서는 HVAC과는 관련없는 정보라고 해줘. 
추론 과정은 나타낼 필요 없어. 최종 답만 나타내.
결과가 빈 리스트면, 답변을 만들지말고, 해당 값이 없다고 답변해줘.
실제 답변을 할때에는, 예시를 그대로 따라하지말고, 변수에 해당하는 수로 바꾸어서 정확하게 답변해야해. 변수명을 쓰는게 아니라 변수에 담긴 값을 확인하고 답변을 해줘. 꼭 중요해. 예시에 담긴 변수명을 그대로 쓴 답변은 하지마.
답변을 할때, 질문을 한번더 쓸 필요는 없어.
결과에 변수에 해당하는 수가 없다면 없다고 표현해야해. 이상한 수를 만들거나 예시를 그대로 따라하지마.
사용자가 언급한 개념(예: "뒷반")이 주어진 메타데이터(`idu_mapping`, `modality_mapping` 등)에서 정확히 어떤 항목을 의미하는지 확인하고,만약 사용자의 질문에서 필요한 정보(예: 특정 장소, 기기, 날짜 범위 등)가 명확하지 않다면, 먼저 해당 정보를 요청해줘. 예를 들어, 사용자가 "뒷반 온도가 가장 높은 날이 언제야?"라고 물었을 때 "뒷반"이 어느 반인지 특정할 수 없으면, `"뒷반이 어디인지 알려주세요."`와 같이 추가 정보를 요청해줘. 
만일 날짜상으로, 미래에 대한 정보를 요청한다면 정보를 제공할 수 없다고 해줘. 추론 하지말고 없는 위치, 미래 날짜 등에 대해서는 정보가 없다고 해줘.
명확하게 메타데이터에 있는 정보가 아니라면 정보가 없다고 해줘. 메타 데이터에 없는 위치는 정보가 없다고하고, 추가 정보를 요청하게 해줘. 
질문하는게 메타데이터 상에서 어떻게 매핑되는지 꼭 확인하고 답변해줘.
"""
input_template = """
질문: {input}
Metadata: {metadata}
예시: {expectations}
결과: {result}
"""
# 결과에는 음수등 의도치 않은 결과가 포함되어 있을 수 있으니, 결과를 왜곡하지 말고 정직하게 답해주세요.

for data in datas:
    start_time = time.time()
    input, tags, scenario, result = data["Input"], data["Tags"], data["Scenario"], data["Result"]
    
    tags = [tags['Type']] if isinstance(tags, dict) else tags
    pass_tags = ["Normal", "Unknown", "Metadata", "NoData", "Unrelated"]
    pass_ = False
    for pass_tag in pass_tags:
        if pass_tag in tags:
            pass_ = True
            break
    if not pass_:
        continue
    
    print("====================================================")
    metadata = metadatas[scenario]

    variables = {}
    for instruction in result:
        i_type = instruction["type"]
        if i_type == "o":
            for k, v in instruction["returns"].items():
                type_, value = v
                # print(k, type_, value)
                if type_ == "pd":
                    # restore pd.DataFrame.to_dict(orient="index") or pd.Series.to_dict()
                    value = pd.DataFrame.from_dict(value, orient="index")
                elif type_ in ["primitive"]:
                    pass
                else:
                    raise NotImplementedError(f"Unsupported type: {k, type_, value}")
                variables[k] = value
            variables.update()

            # print(input, variables)
        elif i_type == "r":
            expectations = instruction["expectations"]
            user_input = input_template.format(
                input=input,
                expectations=expectations,
                result=variables,
                metadata=metadata
            )

            chat = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ]

            print(user_input)

            inputs = tokenizer.apply_chat_template(
                    chat,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt"
            ).to(model.device)

            outputs = model.generate(
                input_ids=inputs,
                max_new_tokens=3000,
                use_cache=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                do_sample=False  # 결정론적 생성
            )
            
            # 결과 디코딩 및 파싱
            responses = tokenizer.batch_decode(outputs, skip_special_tokens=False)[0]

            pattern = r"<\|start_header_id\|>assistant<\|end_header_id\|>(.*?)<\|eot_id\|>"
            match = re.search(pattern, responses, re.DOTALL)
            result = match.group(1).strip() if match else None

            end_time = time.time()
            print("답변:")
            print(result)
            print("Time elapsed:", end_time - start_time)
            



NameError: name 'datas' is not defined